In [ ]:
#######################################################################
##  Made by: Dr. Keungoui Kim
##  Title: Scitech data import from Dimensions
##  goal : 
##  Data set:  
##  Time Span:
##  Variables
##      Input: 
##      Output: 
##  Time-stamp: #  "Sun Jan 26 01:47:34 2020":  edited by awe kim ; code
##  Notice :
#######################################################################

In [1]:
### Library

import pandas as pd
import numpy as np

In [2]:
### Data Import
import dimcli
dimcli.login(key="792DDFAFCCA7478D8F37159F274A2783",
             endpoint="https://app.dimensions.ai/api/dsl/v2")

dsl = dimcli.Dsl()

Dimcli - Dimensions API Client (v0.9.9.1)
Connected to: <https://app.dimensions.ai/api/dsl/v2> - DSL v2.5
Method: manual login


In [156]:
### Extract patent data
pat_data = dsl.query_iterative("""search patents
                                  where publications is not empty and year = 2022                    
                                  return patents[id+year+priority_year+cpc+publications+publication_ids]""") #  limit 50000

Starting iteration with limit=1000 skip=0 ...
0-1000 / 12332 (10.04s)
1000-2000 / 12332 (5.32s)
2000-3000 / 12332 (4.10s)
3000-4000 / 12332 (4.69s)
4000-5000 / 12332 (4.85s)
5000-6000 / 12332 (6.19s)
6000-7000 / 12332 (4.84s)
7000-8000 / 12332 (4.53s)
8000-9000 / 12332 (4.95s)
9000-10000 / 12332 (5.99s)
10000-11000 / 12332 (8.05s)
11000-12000 / 12332 (3.11s)
12000-12332 / 12332 (2.19s)
===
Records extracted: 12332


In [161]:
pat_data_df = pat_data.as_dataframe()
print(pat_data_df.shape)
print(pat_data_df.id.nunique())

(12332, 6)
12332


In [162]:
pat_data_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+"2022"+".csv", index=False)
pat_data_df.head()

,id,cpc,priority_year,publication_ids,publications,year
0,WO-2022236335-A1,"[A61K38/177, A61P29/00, C07K14/70578, A61P37/0...",2021.0,"[pub.1027342753, pub.1009940936]","[{'doi': '10.1182/blood-2003-06-2031', 'id': '...",2022
1,WO-2022236333-A2,"[A61B3/0016, A61B3/0033]",2021.0,[pub.1065228315],"[{'doi': '10.1364/ol.35.000739', 'id': 'pub.10...",2022
2,WO-2022236331-A1,"[B05B12/18, B05B12/16, B05B14/00, B05B7/0408]",2021.0,[pub.1103595211],"[{'doi': '10.1002/adem.201701084', 'id': 'pub....",2022
3,WO-2022236308-A1,"[A61B5/0042, A61B34/30, G01R33/3806, G01R33/34...",2021.0,[pub.1092392383],"[{'doi': '10.1109/tmag.2017.2751001', 'id': 'p...",2022
4,WO-2022236297-A1,"[C11D3/48, C11D3/0068, C11D3/381, C11D7/40, C1...",2021.0,[pub.1080312358],"[{'doi': '10.1128/jb.96.2.479-486.1968', 'id':...",2022


In [25]:
### Import .csv files
# pat_data_df = pd.read_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_01.csv")

# from ast import literal_eval

# pat_data_df['publication_ids'] = pat_data_df['publication_ids'].apply(literal_eval) #convert to list type

In [163]:
pat_pub_df = pat_data_df[['id','publication_ids']]
pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)
pat_pub_df.head()

,id,publication_ids
0,WO-2022236335-A1,pub.1027342753
1,WO-2022236335-A1,pub.1009940936
2,WO-2022236333-A2,pub.1065228315
3,WO-2022236331-A1,pub.1103595211
4,WO-2022236308-A1,pub.1092392383


In [164]:
### Extract publication_ids
pub_list = pat_pub_df.publication_ids.unique()
pd.DataFrame(pub_list, columns=["publication_ids"]).to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+"2022"+".csv")

### Iterative Codes
- Maximum 50,000 records

In [7]:
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] 

for year in years:
    ##### US #####
    query = """search patents 
                where publications is not empty and jurisdiction in ["US"] and priority_year=""" + year + """ 
                return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()

    # Save patent set
    pat_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+year+"_US.csv", index=False)

    # Save publication set
    pat_pub_df = pat_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    # Extract publication_ids
    pub_list = pat_pub_df.publication_ids.unique()
    pd.DataFrame(pub_list, columns=["publication_ids"]).to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+"_US.csv", index=False)

    ##### EP + WO #####
    query = """search patents 
                where publications is not empty and jurisdiction in ["EP","WO"] and priority_year=""" + year + """ 
                return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()

    # Save patent set
    pat_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+year+"_EPWO.csv", index=False)

    # Save publication set
    pat_pub_df = pat_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    # Extract publication_ids
    pub_list = pat_pub_df.publication_ids.unique()
    pd.DataFrame(pub_list, columns=["publication_ids"]).to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+"_EPWO.csv", index=False)


Starting iteration with limit=1000 skip=0 ...
0-1000 / 61554 (3.86s)
1000-2000 / 61554 (4.23s)
2000-3000 / 61554 (5.98s)
3000-4000 / 61554 (6.08s)
4000-5000 / 61554 (4.54s)
5000-6000 / 61554 (4.70s)
6000-7000 / 61554 (4.62s)
7000-8000 / 61554 (4.65s)
8000-9000 / 61554 (4.23s)
9000-10000 / 61554 (4.28s)
10000-11000 / 61554 (4.48s)
11000-12000 / 61554 (6.42s)
12000-13000 / 61554 (5.38s)
13000-14000 / 61554 (3.97s)
14000-15000 / 61554 (3.94s)
15000-16000 / 61554 (6.16s)
16000-17000 / 61554 (5.34s)
17000-18000 / 61554 (4.13s)
18000-19000 / 61554 (3.86s)
19000-20000 / 61554 (3.85s)
20000-21000 / 61554 (4.13s)
21000-22000 / 61554 (6.00s)
22000-23000 / 61554 (4.08s)
23000-24000 / 61554 (4.03s)
24000-25000 / 61554 (4.22s)
25000-26000 / 61554 (4.64s)
26000-27000 / 61554 (4.18s)
27000-28000 / 61554 (3.95s)
28000-29000 / 61554 (6.12s)
29000-30000 / 61554 (5.85s)
30000-31000 / 61554 (3.92s)
31000-32000 / 61554 (6.11s)
32000-33000 / 61554 (6.26s)
33000-34000 / 61554 (4.09s)
34000-35000 / 61554 (3.9

In [7]:
### Check total counts

years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] 
totalRecUS = []
totalRecEUWO = []
for year in years:
    query = """search patents 
                    where publications is not empty and jurisdiction in ["US"] and priority_year=""" + year + """ 
                    return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query(query) 
    totalRecUS.append(pat_data.count_total)

    query = """search patents 
            where publications is not empty and jurisdiction in ["EP","WO"] and priority_year=""" + year + """ 
            return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query(query) 
    totalRecEUWO.append(pat_data.count_total)

    print(year)

# Present result
pd.DataFrame({'year':years,'USRec':totalRecUS,'EUWORec':totalRecEUWO})

Returned Patents: 20 (total = 61554)
Time: 1.48s
Returned Patents: 20 (total = 42638)
Time: 6.01s
2010
Returned Patents: 20 (total = 64011)
Time: 1.35s
Returned Patents: 20 (total = 42616)
Time: 4.70s
2011
Returned Patents: 20 (total = 65138)
Time: 1.53s
Returned Patents: 20 (total = 40043)
Time: 4.44s
2012
Returned Patents: 20 (total = 67394)
Time: 6.15s
Returned Patents: 20 (total = 42498)
Time: 1.47s
2013
Returned Patents: 20 (total = 61421)
Time: 4.50s
Returned Patents: 20 (total = 39677)
Time: 1.48s
2014
Returned Patents: 20 (total = 57035)
Time: 4.53s
Returned Patents: 20 (total = 38526)
Time: 1.48s
2015
Returned Patents: 20 (total = 50138)
Time: 6.03s
Returned Patents: 20 (total = 40015)
Time: 1.53s
2016
Returned Patents: 20 (total = 43624)
Time: 5.98s
Returned Patents: 20 (total = 42727)
Time: 1.50s
2017
Returned Patents: 20 (total = 30421)
Time: 1.85s
Returned Patents: 20 (total = 46209)
Time: 5.65s
2018
Returned Patents: 20 (total = 19572)
Time: 1.55s
Returned Patents: 20 (to

#### Alternative codes

In [ ]:
years = '2021'

for year in years:
    query = """search patents 
            where publications is not empty and year=""" + year + """
            return patents[id+year+priority_year+cpc+publications+publication_ids]"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()
    rangeIndex = int(temp.count_total/1000)+1

    for t in range(50,rangeIndex):
        query = """search patents 
            where publications is not empty and year=""" + year + """
            return patents[id+year+priority_year+cpc+publications+publication_ids+reference_ids+times_cited] limit 1000 skip """ + str(1000*t)
        temp_data = dsl.query(query)
        temp_df = temp_data.as_dataframe()
        # bind rows
        pat_df = pd.concat([pat_df, temp_df], axis=0)
        print(t)

    # Save patent set
    pat_data_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+year+".csv", index=False)

    # Save publication set
    pat_pub_df = pat_data_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    # Extract publication_ids
    pub_list = pat_pub_df.publication_ids.unique()
    pd.DataFrame(pub_list, columns=["publication_ids"]).to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+".csv", index=False)

In [55]:
%%dslloop my_data search publications for "malaria" return publications limit 500


UsageError: %%dslloop is a cell magic, but the cell body is empty. Did you mean the line magic %dslloop (single %)?
